# Cистемы прогнозирования качества молока и объем удоя коров для покупки их молочному хозяйству «Вольный луг»

содержание

_____
_____
## Описание

Молочное хозяйство «Вольный луг», ориентированное на расширение поголовья коров, заключило контракт с ассоциацией пастбищ «ЭкоФерма». Фермер стремится создать стабильное и высококачественное стадо, отбирая бурёнок, которые не только обеспечат высокий удой, но и удовлетворят строгим требованиям к вкусовым качествам молока. Для этого требуется система, которая поможет автоматизировать процесс отбора, снижая риски и обеспечивая объективность в принятии решения.

- Цели и задачи проекта:

    - Прогнозирование удоя коровы: Разработка модели машинного обучения, которая на основе данных о корове (возраст, порода, физическое состояние, рацион, генетика и т.д.) будет прогнозировать её потенциальный удой за год. Модель должна обеспечивать точность предсказания с учётом различных факторов, влияющих на продуктивность.

    - Оценка качества молока:
    Вторая модель будет прогнозировать вероятность того, что молоко от данной коровы будет соответствовать требованиям фермерского хозяйства по вкусовым качествам. Это может включать параметры, такие как жирность молока, соотношение белков и жиров, а также вкусовые характеристики, оцененные по шкале фермером.

    - Критерии отбора коров:
    Для фермерского хозяйства критически важно, чтобы каждая покупаемая бурёнка:
        - Обеспечивала удой не менее 6000 кг молока в год.
        - Производила молоко, соответствующее строгим критериям вкуса.

_____
_____
## Импорт и подготовка к работе

In [1]:
# %pip install bidict
# %pip install phik

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from bidict import bidict
from sklearn.metrics import r2_score
from plotly.colors import find_intermediate_color, hex_to_rgb
from phik import resources
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from itertools import cycle

In [3]:
pd.set_option('display.max_columns', None)

_____
_____
## Загрузка данных, общая информация и первичная обработка

_____
### Скачивание датасетов и общая информация

1. **```ferma_main.csv```**

|Поле            |Описание                                                           |
|----------------|-------------------------------------------------------------------|
|id              |Уникальный идентификатор коровы                                    |
|Удой, кг        |Масса молока, которую корова даёт в год (в килограммах)            |
|ЭКЕ             |Показатель питательности корма коровы                              |
|Сырой протеин, г|Содержание сырого протеина в корме (в граммах)                     |
|СПО             |Отношение сахара к протеину в корме коровы                         |
|Порода          |Порода коровы                                                      |
|Тип пастбища    |Ландшафт лугов, на которых паслась корова                          |
|порода папы_быка|Порода папы коровы                                                 |
|Жирность, %     |Содержание жиров в молоке (в процентах)                            |
|Белок, %        |Содержание белков в молоке (в процентах)                           |
|Вкус молока     |Оценка вкуса молока по личным критериям фермера (вкусно, не вкусно)|
|Возраст         |Возраст коровы (менее 2 лет, более 2 лет)                          |

In [4]:
ferma = pd.read_csv('ferma_main.csv', delimiter=';', decimal=',')
display(ferma.head(10))
ferma.info()

,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст
0,1,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет
1,2,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет
2,3,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет
3,4,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет
4,5,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет
5,6,5254,12.7,1806,0.885,РефлешнСоверинг,Холмистое,Соверин,3.13,3.078,вкусно,менее_2_лет
6,7,5910,13.2,1792,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.48,3.077,вкусно,более_2_лет
7,8,6179,14.7,1806,0.930,Вис Бик Айдиал,Равнинные,Айдиал,3.55,3.076,вкусно,более_2_лет
8,9,6783,14.2,1918,0.940,РефлешнСоверинг,Холмистое,Айдиал,3.71,3.077,вкусно,более_2_лет
9,10,5339,13.4,1791,0.895,Вис Бик Айдиал,Холмистое,Соверин,3.13,3.075,вкусно,менее_2_лет


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     634 non-null    int64  
 1   Удой, кг                               634 non-null    int64  
 2   ЭКЕ (Энергетическая кормовая единица)  634 non-null    float64
 3   Сырой протеин, г                       634 non-null    int64  
 4   СПО (Сахаро-протеиновое соотношение)   634 non-null    float64
 5   Порода                                 634 non-null    object 
 6   Тип пастбища                           634 non-null    object 
 7   порода папы_быка                       634 non-null    object 
 8   Жирность,%                             634 non-null    float64
 9   Белок,%                                634 non-null    float64
 10  Вкус молока                            634 non-null    object 
 11  Возрас

2. **```ferma_dad.csv```**

|Поле    |Описание                       |
|--------|-------------------------------|
|id      |Уникальный идентификатор коровы|
|Имя Папы|Имя папы коровы                |

In [5]:
dads = pd.read_csv('ferma_dad.csv', delimiter=';')
display(dads.head())
dads.info()

,id,Имя Папы
0,1,Буйный
1,2,Соловчик
2,3,Барин
3,4,Буйный
4,5,Барин


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        629 non-null    int64 
 1   Имя Папы  629 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.0+ KB


3. **```interested_cows.csv```**

|Поле                   |Описание                                                   |
|-----------------------|-----------------------------------------------------------|
|Порода                 |Порода коровы                                              |
|Тип пастбища           |Ландшафт лугов, на которых паслась корова                  |
|порода папы_быка       |Порода папы коровы                                         |
|Имя_папы               |Имя папы коровы                                            |
|Текущая_жирность,%     |Содержание жиров в молоке (в процентах)                    |
|Текущий_уровень_белок,%|Содержание белков в молоке (в процентах)                   |
|Возраст                |Возраст коровы, бинарный признак (менее_2_лет, более_2_лет)|


In [6]:
interested_cows = pd.read_csv('cow_buy.csv', delimiter=';', decimal=',')
display(interested_cows.head())
interested_cows.info()

,Порода,Тип пастбища,порода папы_быка,Имя_папы,"Текущая_жирность,%","Текущий_уровень_белок,%",Возраст
0,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,3.58,3.076,более_2_лет
1,Вис Бик Айдиал,равнинное,Соверин,Буйный,3.54,3.081,менее_2_лет
2,РефлешнСоверинг,равнинное,Соверин,Барин,3.59,3.074,более_2_лет
3,РефлешнСоверинг,холмистое,Айдиал,Буйный,3.40,3.061,более_2_лет
4,РефлешнСоверинг,равнинное,Айдиал,Буйный,3.64,3.074,более_2_лет


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Порода                   20 non-null     object 
 1   Тип пастбища             20 non-null     object 
 2   порода папы_быка         20 non-null     object 
 3   Имя_папы                 20 non-null     object 
 4   Текущая_жирность,%       20 non-null     float64
 5   Текущий_уровень_белок,%  20 non-null     float64
 6   Возраст                  20 non-null     object 
dtypes: float64(2), object(5)
memory usage: 1.2+ KB


_____
### Первичная обработка

Объединим датасеты **```dads```** и **```ferma```**

In [7]:
ferma = pd.merge(ferma, dads, on='id', how='left')
ferma.head()

,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст,Имя Папы
0,1,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет,Буйный
1,2,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет,Соловчик
2,3,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет,Барин
3,4,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет,Буйный
4,5,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет,Барин


_____
### Вывод:

- ```ferma``` (```ferma_main.csv```)
    - Содержит данные о стаде фермера. Включает информацию о коровах, их породах, питании, молочной продуктивности и вкусе молока. Структура данных позволяет анализировать взаимосвязь между кормом, породой, возрастом и качеством молока.
    - Всего 634 строки.
    - Нет пропущенных значений.
    - 3 количественных признака: ```Удой, кг```; ```Сырой протеин, г```; ```Жирность, %```, ```Белок, %```.
    - 9 категориальных признаков: ```ЭКЕ```, ```СПО```, ```Порода```, ```Тип пастбища```, ```Порода папы_быка```, ```Вкус молока```, ```Возраст```.
    - Типы данных верные.
    - Надо переименовать имена столбцов.

- ```dads``` (```ferma_dad.csv```) - Объединил с ```ferma``` по ```id```
    - Хранит имя папы каждой коровы. Этот датасет позволяет связать коров с их отцами для дальнейшего анализа генетических характеристик.
    - Всего 629 строк.
    - Нет пропущенных значений.
    - 1 количественный признак: ```id```.
    - 1 категориальный признак: ```Имя Папы```.
    - Типы данных верные.
    - Надо переименовать имена столбцов.

- ```interested_cows``` (```cow_buy.csv```)
    - Содержит данные о коровах, которых фермер планирует купить. Включает информацию о породе, пастбище, кормлении и молочной продуктивности на момент продажи, но без данных о корме и удое.
    - Всего 20 строк.
    - Нет пропущенных значений.
    - 0 количественных признаков.
    - 7 категориальных признаков: ```Порода```, ```Тип пастбища```, ```порода папы_быка```, ```Имя_папы```, ```Текущая_жирность,%```, ```Текущий_уровень_белок,%```, ```Возраст```.
    - Типы данных верные.
    - Надо переименовать имена столбцов.

_____
_____
## Предобработка данных

_____
### Удаление ненужного столбца id

Так как я слил два датасета по ```id```, то этот признак больше не нужен

In [8]:
ferma = ferma.drop('id', axis=1)
ferma.head()

,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст,Имя Папы
0,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет,Буйный
1,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет,Соловчик
2,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет,Барин
3,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет,Буйный
4,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет,Барин


_____
### Переименовывание столбцов датафреймов

1. **```ferma```**

In [9]:
ferma_columns_translate = bidict({
    'Удой, кг': 'milk_yield',
    'ЭКЕ (Энергетическая кормовая единица)': 'energy_unit',
    'Сырой протеин, г': 'raw_protein',
    'СПО (Сахаро-протеиновое соотношение)': 'sugar_protein_ratio',
    'Порода': 'breed',
    'Тип пастбища': 'pasture_type',
    'порода папы_быка': 'dad_breed',
    'Жирность,%': 'fat_content',
    'Белок,%': 'protein_content',
    'Вкус молока': 'milk_taste',
    'Возраст': 'age',
    'Имя Папы': 'dad_name'
})

ferma = ferma.rename(columns=ferma_columns_translate)
ferma.head()

,milk_yield,energy_unit,raw_protein,sugar_protein_ratio,breed,pasture_type,dad_breed,fat_content,protein_content,milk_taste,age,dad_name
0,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет,Буйный
1,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет,Соловчик
2,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет,Барин
3,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет,Буйный
4,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет,Барин


|Поле               |Описание                                                           |
|-----------------  |-------------------------------------------------------------------|
|milk_yield         |Масса молока, которую корова даёт в год (в килограммах)            |
|energy_unit        |Показатель питательности корма коровы                              |
|raw_protein        |Содержание сырого протеина в корме (в граммах)                     |
|sugar_protein_ratio|Отношение сахара к протеину в корме коровы                         |
|breed              |Порода коровы                                                      |
|pasture_type       |Ландшафт лугов, на которых паслась корова                          |
|dad_breed          |Порода папы коровы                                                 |
|fat_content        |Содержание жиров в молоке (в процентах)                            |
|protein_content    |Содержание белков в молоке (в процентах)                           |
|milk_taste         |Оценка вкуса молока по личным критериям фермера (вкусно, не вкусно)|
|age                |Возраст коровы (менее 2 лет, более 2 лет)                          |
|dad_name           |Имя папы коровы                                                    |

2. **```interested_cows```**

In [10]:
interested_cows_columns_translate = bidict({
    'Порода': 'breed',
    'Тип пастбища': 'pasture_type',
    'порода папы_быка': 'dad_breed',
    'Имя_папы': 'dad_name',
    'Текущая_жирность,%': 'current_fat_content',
    'Текущий_уровень_белок,%': 'current_protein_content',
    'Возраст': 'age'
})

interested_cows = interested_cows.rename(columns=interested_cows_columns_translate)
interested_cows.head()

,breed,pasture_type,dad_breed,dad_name,current_fat_content,current_protein_content,age
0,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,3.58,3.076,более_2_лет
1,Вис Бик Айдиал,равнинное,Соверин,Буйный,3.54,3.081,менее_2_лет
2,РефлешнСоверинг,равнинное,Соверин,Барин,3.59,3.074,более_2_лет
3,РефлешнСоверинг,холмистое,Айдиал,Буйный,3.40,3.061,более_2_лет
4,РефлешнСоверинг,равнинное,Айдиал,Буйный,3.64,3.074,более_2_лет


|Поле                   |Описание                                  |
|-----------------------|------------------------------------------|
|breed                  |Порода коровы                             |
|pasture_type           |Ландшафт лугов, на которых паслась корова |
|dad_breed              |Порода папы коровы                        |
|dad_name               |Имя папы коровы                           |
|current_fat_content    |Содержание жиров в молоке (в процентах)   |
|current_protein_content|Содержание белков в молоке (в процентах)  |
|age                    |Возраст коровы (менее 2 лет, более 2 лет) |

_____
### Обработка дубликатов

- ***Явные***

1. **```ferma```**

In [11]:
print('Кол-во явных дубликатов:', ferma.duplicated().sum())
ferma = ferma.drop_duplicates()

Кол-во явных дубликатов: 5


2. **```interested_cows```**

In [12]:
print('Кол-во явных дубликатов:', interested_cows.duplicated().sum())
interested_cows = interested_cows.drop_duplicates()

Кол-во явных дубликатов: 4


- ***Неявные***

1. **```ferma```**

In [13]:
for categorical_col in ferma.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{ferma[categorical_col].unique()}')

Уникальные значения breed       	:	['Вис Бик Айдиал' 'РефлешнСоверинг']
Уникальные значения pasture_type	:	['Равнинное' 'Равнинные' 'Холмистое']
Уникальные значения dad_breed   	:	['Айдиал' 'Соверин' 'Айдиалл']
Уникальные значения milk_taste  	:	['вкусно' 'не вкусно']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']
Уникальные значения dad_name    	:	['Буйный' 'Соловчик' 'Барин' 'Геркулес']


- В столбце ```dad_breed``` есть неявный дубликат ("Айдиал" и "Айдиалл")

In [14]:
ferma.dad_breed = ferma.dad_breed.replace('Айдиалл', 'Айдиал')
print(f'Уникальные значения dad_breed: {ferma.dad_breed.unique()}')

Уникальные значения dad_breed: ['Айдиал' 'Соверин']


- В столбце ```pasture_type``` есть неявный дубликат ("Равнинное" и "Равнинные")

In [15]:
ferma.pasture_type = ferma.pasture_type.replace('Равнинные', 'Равнинное')
print(f'Уникальные значения pasture_type: {ferma.pasture_type.unique()}')

Уникальные значения pasture_type: ['Равнинное' 'Холмистое']


2. **```interested_cows```**

In [16]:
for categorical_col in interested_cows.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{interested_cows[categorical_col].unique()}')

Уникальные значения breed       	:	['Вис Бик Айдиал' 'РефлешнСоверинг']
Уникальные значения pasture_type	:	['холмистое' 'равнинное']
Уникальные значения dad_breed   	:	['Айдиал' 'Соверин']
Уникальные значения dad_name    	:	['Геркулес' 'Буйный' 'Барин' 'Соловчик']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']


Неявные дубликаты отсутствуют

3. **```interested_cows```** + **```ferma```**

In [17]:
print('ferma:')
for categorical_col in ferma.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{ferma[categorical_col].unique()}')
print('\n\ninterested_cows:')
for categorical_col in interested_cows.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{interested_cows[categorical_col].unique()}')

ferma:
Уникальные значения breed       	:	['Вис Бик Айдиал' 'РефлешнСоверинг']
Уникальные значения pasture_type	:	['Равнинное' 'Холмистое']
Уникальные значения dad_breed   	:	['Айдиал' 'Соверин']
Уникальные значения milk_taste  	:	['вкусно' 'не вкусно']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']
Уникальные значения dad_name    	:	['Буйный' 'Соловчик' 'Барин' 'Геркулес']


interested_cows:
Уникальные значения breed       	:	['Вис Бик Айдиал' 'РефлешнСоверинг']
Уникальные значения pasture_type	:	['холмистое' 'равнинное']
Уникальные значения dad_breed   	:	['Айдиал' 'Соверин']
Уникальные значения dad_name    	:	['Геркулес' 'Буйный' 'Барин' 'Соловчик']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']


Хоть датафреймы разные, некоторые их столбцы отвечают за одни признаки, и разные наименования одной категории в разных датафреймах тоже можно считать неявным дубликатом. Если мы занизим все символы в категориальных фичах, то придем к единому именованию.

In [18]:
ferma = ferma.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)
interested_cows = interested_cows.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

print('ferma:')
for categorical_col in ferma.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{ferma[categorical_col].unique()}')
print('\n\ninterested_cows:')
for categorical_col in interested_cows.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{interested_cows[categorical_col].unique()}')

ferma:
Уникальные значения breed       	:	['вис бик айдиал' 'рефлешнсоверинг']
Уникальные значения pasture_type	:	['равнинное' 'холмистое']
Уникальные значения dad_breed   	:	['айдиал' 'соверин']
Уникальные значения milk_taste  	:	['вкусно' 'не вкусно']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']
Уникальные значения dad_name    	:	['буйный' 'соловчик' 'барин' 'геркулес']


interested_cows:
Уникальные значения breed       	:	['вис бик айдиал' 'рефлешнсоверинг']
Уникальные значения pasture_type	:	['холмистое' 'равнинное']
Уникальные значения dad_breed   	:	['айдиал' 'соверин']
Уникальные значения dad_name    	:	['геркулес' 'буйный' 'барин' 'соловчик']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']


_____
### Обработка аномальных значений

1. **```ferma```**

In [19]:
fig = make_subplots(
    rows=2, cols=3,
    vertical_spacing=0.07,
    subplot_titles=[
        ferma_columns_translate.inverse[name] for name in ferma.select_dtypes(include=['number']).columns
    ])

for i, column in enumerate(ferma.select_dtypes(include=['number']).columns):
    fig.add_trace(
        go.Box(y=ferma[column], name=''),
        row=(i // 3) + 1, col=(i % 3) + 1
    )
fig.update_layout(height=1000, width=1000, showlegend=False, title='График "Ящик с усами" для каждой количественной фичи в ferma')

"В среднем, корова может дать от 5 000 до 10 000 литров молока в год в зависимости от породы, рациона и условий содержания. Однако, более продуктивные породы могут достигать удоя до 12 000-15 000 литров в год."
- Выбросом можно считать только значение 45616 в столбце ```Удой, кг``` (```milk_yield```). 

In [20]:
display(ferma[ferma.milk_yield > 15000])
ferma = ferma[ferma.milk_yield < 15000]

,milk_yield,energy_unit,raw_protein,sugar_protein_ratio,breed,pasture_type,dad_breed,fat_content,protein_content,milk_taste,age,dad_name
16,45616,11.5,1675,0.93,вис бик айдиал,холмистое,айдиал,3.22,3.076,вкусно,более_2_лет,буйный


- Также математически слишком аномальными можно считать коров, у которых ```Жирность,%``` (```fat_content```) < 3.1

In [21]:
display(ferma[ferma.fat_content < 3.1])
ferma = ferma[ferma.fat_content >= 3.1]

,milk_yield,energy_unit,raw_protein,sugar_protein_ratio,breed,pasture_type,dad_breed,fat_content,protein_content,milk_taste,age,dad_name
17,5777,12.4,1873,0.895,вис бик айдиал,холмистое,айдиал,3.03,3.080,вкусно,более_2_лет,буйный
554,5686,14.7,1712,0.860,рефлешнсоверинг,холмистое,айдиал,3.01,3.074,вкусно,более_2_лет,буйный
558,5346,12.4,1837,0.860,рефлешнсоверинг,холмистое,соверин,3.01,3.078,не вкусно,менее_2_лет,барин
561,5300,13.0,1785,0.860,вис бик айдиал,равнинное,соверин,2.99,3.077,не вкусно,менее_2_лет,соловчик
568,5989,13.2,1721,0.860,вис бик айдиал,холмистое,соверин,2.99,3.076,вкусно,менее_2_лет,соловчик
576,5376,11.7,1825,0.860,рефлешнсоверинг,равнинное,соверин,2.99,3.076,не вкусно,менее_2_лет,соловчик


- Выбросы, которые отображены на графике признака ```Белок,%``` (```protein_content```), трогать не будем, потому что они вполне укладываются в рамки разумного.

2. **```interested_cows```**

In [22]:
fig = make_subplots(
    rows=1, cols=2,
    vertical_spacing=0.07,
    subplot_titles=[
        interested_cows_columns_translate.inverse[name] for name in interested_cows.select_dtypes(include=['number']).columns
    ])

for i, column in enumerate(interested_cows.select_dtypes(include=['number']).columns):
    fig.add_trace(
        go.Box(y=interested_cows[column], name=''),
        row=(i // 3) + 1, col=(i % 3) + 1
    )
fig.update_layout(height=550, width=750, showlegend=False, title='График "Ящик с усами" для каждой количественной фичи в interested_cows')

Выбросов нет.

_____
### Общая обработка данных

- В ```interested_cows``` параметры корма ```ЭКЕ (Энергетическая кормовая единица)```, ```Сырой протеин, г``` и ```СПО (Сахаро-протеиновое соотношение)``` отсутствуют. Технологи заказчика пересмотрели подход к кормлению: для новых коров планируется увеличить значения каждого из этих параметров на 5%. Поэтому добавим эти три признака, взяв данные из ```ferma```.

In [23]:
interested_cows['energy_unit'] = ferma.energy_unit.mean() * 1.05
interested_cows['raw_protein'] = ferma.raw_protein.mean() * 1.05
interested_cows['sugar_protein_ratio'] = ferma.sugar_protein_ratio.mean() * 1.05
interested_cows_columns_translate = bidict({**interested_cows_columns_translate,
                                            **{'ЭКЕ (Энергетическая кормовая единица)': 'energy_unit',
                                               'Сырой протеин, г': 'raw_protein',
                                               'СПО (Сахаро-протеиновое соотношение)': 'sugar_protein_ratio'}})
interested_cows.head()

,breed,pasture_type,dad_breed,dad_name,current_fat_content,current_protein_content,age,energy_unit,raw_protein,sugar_protein_ratio
0,вис бик айдиал,холмистое,айдиал,геркулес,3.58,3.076,более_2_лет,15.292862,2021.280386,0.959222
1,вис бик айдиал,равнинное,соверин,буйный,3.54,3.081,менее_2_лет,15.292862,2021.280386,0.959222
2,рефлешнсоверинг,равнинное,соверин,барин,3.59,3.074,более_2_лет,15.292862,2021.280386,0.959222
3,рефлешнсоверинг,холмистое,айдиал,буйный,3.40,3.061,более_2_лет,15.292862,2021.280386,0.959222
4,рефлешнсоверинг,равнинное,айдиал,буйный,3.64,3.074,более_2_лет,15.292862,2021.280386,0.959222


_____
### Вывод:

- ```ferma```:
    - Кол-во строк в сократилось с 634 до 628.
    - Удален столбец ```id```, из-за дальнейшей ненадобности.
    - Переименованы столбцы на корректные имена.
    - Обработаны дубликаты и аномальные значения.

|Старые имена        |Новые имена          |Описание                                                           |
|--------------------|---------------------|-------------------------------------------------------------------|
|~~id~~              |**-**                |Уникальный идентификатор коровы                                    |
|~~Удой, кг~~        |*milk_yield*         |Масса молока, которую корова даёт в год (в килограммах)            |
|~~ЭКЕ~~             |*energy_unit*        |Показатель питательности корма коровы                              |
|~~Сырой протеин, г~~|*raw_protein*        |Содержание сырого протеина в корме (в граммах)                     |
|~~СПО~~             |*sugar_protein_ratio*|Отношение сахара к протеину в корме коровы                         |
|~~Порода~~          |*breed*              |Порода коровы                                                      |
|~~Тип пастбища~~    |*pasture_type*       |Ландшафт лугов, на которых паслась корова                          |
|~~порода папы_быка~~|*dad_breed*          |Порода папы коровы                                                 |
|~~Жирность, %~~     |*fat_content*        |Содержание жиров в молоке (в процентах)                            |
|~~Белок, %~~        |*protein_content*    |Содержание белков в молоке (в процентах)                           |
|~~Вкус молока~~     |*milk_taste*         |Оценка вкуса молока по личным критериям фермера (вкусно, не вкусно)|
|~~Возраст~~         |*age*                |Возраст коровы (менее 2 лет, более 2 лет)                          |
|**-**               |*dad_name*           |Имя папы коровы                                                    |

- ```interested_cows```:
    - Кол-во строк в сократилось с 20 до 18.
    - Переименованы столбцы на корректные имена.
    - Обработаны дубликаты.
    - Добавлены признаки ```energy_unit```, ```raw_protein```, ```sugar_protein_ratio``` по пересмотренному технологами подходу к кормлению коров

|Старые имена               |Новые имена              |Описание                                      |
|---------------------------|-------------------------|----------------------------------------------|
|~~Порода~~                 |*breed*                  |Порода коровы                                 |
|~~Тип пастбища~~           |*pasture_type*           |Ландшафт лугов, на которых паслась корова     |
|~~порода папы_быка~~       |*dad_breed*              |Порода папы коровы                            |
|~~Имя_папы~~               |*dad_name*               |Имя папы коровы                               |
|~~Текущая_жирность,%~~     |*current_fat_content*    |Содержание жиров в молоке (в процентах)       |
|~~Текущий_уровень_белок,%~~|*current_protein_content*|Содержание белков в молоке (в процентах)      |
|~~Возраст~~                |*age*                    |Возраст коровы (менее 2 лет, более 2 лет)     |
|**-**                      |*energy_unit*            |Показатель питательности корма коровы         |
|**-**                      |*raw_protein*            |Содержание сырого протеина в корме (в граммах)|
|**-**                      |*sugar_protein_ratio*    |Отношение сахара к протеину в корме коровы    |

_____
_____
## Анализ данных

_____
### Функции отрисовок данных

In [24]:
def statistical_graphis_for_categorical(data, hist_title, agg='val_count', top_n_in_pie=5):
    fig = make_subplots(
        rows=1, cols=2, specs=[[{'type': 'xy'}, {'type': 'domain'}]],
        subplot_titles=('Гистограмма', 'Круговая диаграмма')
    )
    if agg == 'val_count':
        category_agg = data.value_counts()
    elif agg == 'sum':
        category_agg = data.sum()
    else:
        raise ValueError('agg="val_count"|sum')
    categories = category_agg.index
    val_agg = category_agg.values

    fig.add_trace(
        go.Bar(
            x=val_agg, y=categories, 
            orientation='h',
            marker_color='green', 
			showlegend=False, 
			name=data.name,
            width=0.8
        ),
        row=1, col=1
    )
    fig.update_xaxes(title_text='Частота' if agg == 'val_count' else 'Сумма', row=1, col=1)
    fig.update_yaxes(title_text=hist_title, tickvals=categories, row=1, col=1)

    other = [category_agg[top_n_in_pie:].sum()]
    fig.add_trace(
        go.Pie(labels=category_agg.head(top_n_in_pie).index.tolist() + (['Остальные'] if other[0] else []),
               values=category_agg.head(top_n_in_pie).values.tolist() + (other if other[0] else []),
               name='',
               textinfo='label+percent'),
        row=1, col=2
    )

    fig.update_layout(
        title_text=f'Статистические графики по колонке <b>{hist_title}</b><br>(<b>{data.name}</b>)',
        title_x=0.5,
        showlegend=True,
        width=1000,
        height=600,
    )

    fig.show()


def statistical_graphis_for_numeric(col: pd.Series, xaxis_title, nbinsx=50):
	fig = make_subplots(rows=1, cols=2, subplot_titles=('Гистограмма', 'Диаграмма размаха'))

	fig.add_trace(
		go.Histogram(x=col, nbinsx=nbinsx, marker_color='blue', name=col.name),
		row=1, col=1
	)
	fig.update_xaxes(title_text=xaxis_title, row=1, col=1)
	fig.update_yaxes(title_text='Частота', row=1, col=1)

	fig.add_trace(
		go.Box(y=col, marker_color='orange', name=''),
		row=1, col=2
	)
	fig.update_yaxes(title_text=xaxis_title, row=1, col=2)

	fig.update_layout(
		title_text=f'Статистические графики по значению <b>{xaxis_title}</b><br>(<b>{col.name}</b>)',
        title_x=0.5,
		showlegend=False,
		width=1000,
		height=500
	)

	fig.show()

_____
### Количественные признаки

In [25]:
for num_col in ferma.select_dtypes(include=[int, float]).columns:
    statistical_graphis_for_numeric(
        col=ferma[num_col],
        xaxis_title=ferma_columns_translate.inverse[num_col],
        nbinsx={'milk_yield': 25,
                'energy_unit': 20,
                'raw_protein': 30,
                'sugar_protein_ratio': 20,
                'fat_content': 10,
                'protein_content': 30}[num_col]
    )

- ```Удой, кг```: Гистограмма описывается нормальным распределением.
- ```ЭКЕ (Энергетическая кормовая единица)```: Гистограмма описывает распределение Пуассона. 
- ```Сырой протеин, г```: Гистограмма имеет нисходящий тренд, что говорит о том что коров с более низким отношением сахара к протеину преимущественно.
- ```СПО (Сахаро-протеиновое соотношение)```: Многомодальное распределение, что может говорить о наличии подгрупп в данных. 
- ```Жирность,%```: Гистограма имеет высходящий тренд, что говорит о том, что коровы с жирным молокос преобладают.
- ```Белок,%```: Гистограмма описывается нормальным распределением, что есть хорошо для количественного признака.

_____
### Категориальные признаки

In [26]:
for cat_col in ferma.select_dtypes(include='object').columns:
    statistical_graphis_for_categorical(ferma[cat_col], hist_title=ferma_columns_translate.inverse[cat_col], top_n_in_pie=5)

- Могу сказать только то, что нет дисбаланса классов категориальных признаков, что есть хорошо.

_____
### Вывод:

- Анализ количественных признаков показал разные типы распределений: нормальное (удой, белок) до пуассоновское (ЭКЕ) и многомодального (СПО). Отмечено преобладание низкого содержания сырого протеина и высокожирного молока, что отражает особенности коров.
- Категориальные признаки сбалансированы по классам, что положительно влияет на дальнейший анализ.
- Возможно стоит обратить внимание на значение СПО, так как оно имеет нестандратное распределение (можно расмотреть возможность написания ансамбля бэггинг).

_____
_____
## Корреляционный анализ признаков

_____
### Матрица корреляции

In [27]:
px.imshow(
    ferma.rename(columns=ferma_columns_translate.inv).phik_matrix(), 
    text_auto='.3f',
    color_continuous_scale='oranges',
    title='Матрица корреляции для <b>ferma</b>'
).update_layout(
    width=1000,
    height=800,
    title_font_size=20,
    font=dict(size=14)
)

interval columns not set, guessing: ['Удой, кг', 'ЭКЕ (Энергетическая кормовая единица)', 'Сырой протеин, г', 'СПО (Сахаро-протеиновое соотношение)', 'Жирность,%', 'Белок,%']


С нашим таргетом (```Удой,кг```) коррелируют:
- 2 признака средне (>0.5): ```СПО (Сахаро-протеиновое соотношение)```, ```порода папы_быка```.
- 3 признаков сильно (>0.7): ```Возраст```, ```ЭКЕ (Энергетическая кормовая единица)```, ```Жирность,%```.

Следует отметить наличие мультиколлинеарности, к примеру: 
- corr(```Возраст```, ```ЭКЕ...```) = 0.767
- corr(```ЭКЕ...```, ```Жирность,%```) = 0.723
- corr(```Возраст```, ```СПО...```) = 0.672

_____
### Матрица значимости

Эта матрица показывает, какие коэффициенты корреляции являются статистически значимыми.

In [28]:
px.imshow(
    ferma.rename(columns=ferma_columns_translate.inv).significance_matrix(), 
    text_auto='.3f',
    color_continuous_scale='oranges',
    title='Матрица значимости для <b>ferma</b>'
).update_layout(
    width=1000,
    height=800,
    title_font_size=20,
    font=dict(size=14)
)

interval columns not set, guessing: ['Удой, кг', 'ЭКЕ (Энергетическая кормовая единица)', 'Сырой протеин, г', 'СПО (Сахаро-протеиновое соотношение)', 'Жирность,%', 'Белок,%']


- Все вышесказанные признаки имеют аналогично высокие значения и в матрице значимости, кроме признака ```порода папы_быка```

_____
### Диаграмма рассеяния с разбиением на категории. 

In [29]:
color_palette = cycle(px.colors.qualitative.Plotly)

def draw_scatter_between_milk_yield_ans(num_col, opacity=0.4):

    fig = make_subplots(
        rows=3, cols=2, vertical_spacing=0.09,
        subplot_titles=[ferma_columns_translate.inverse[col] for col in ferma.select_dtypes(include='object').columns]
    )

    for cetegoty_i, category_col in enumerate(ferma.select_dtypes(include='object').columns):
        # Перебор категориальных признаков

        for class_of_category, group_of_category in ferma.groupby(category_col):
            # Перебор по каждому классу категории

            color = next(color_palette)
            x = group_of_category[num_col]
            y = group_of_category['milk_yield']

            # диаграмма рассеяния по классу
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y,
                    mode='markers',
                    marker=dict(size=7, opacity=opacity, color=color),
                    name=class_of_category,
                    legendgroup=ferma_columns_translate.inverse[category_col],
                    legendgrouptitle=dict(text=ferma_columns_translate.inverse[category_col]),
                ),
                row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1
            )
            slope, intercept = np.polyfit(x, y, 1)
            trend_line = slope * x + intercept

            # Линия тренда по классу
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=trend_line,
                    mode='lines',
                    line=dict(
                        color=f'rgb{find_intermediate_color(hex_to_rgb(color), [0] * 3, 0.4)}',
                        width=3
                    ),
                    name='Линия тренда',
                    visible='legendonly',
                    legendgroup=ferma_columns_translate.inverse[category_col],
                ),
                row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1
            )

        # Общая линия тренда
        slope, intercept = np.polyfit(ferma[num_col], ferma.milk_yield, 1)
        trend_line = slope * ferma[num_col] + intercept
        fig.add_trace(
            go.Scatter(
                x=ferma[num_col],
                y=trend_line,
                mode='lines',
                name='Общая линия тренда',
                visible='legendonly',
                line=dict(color='black', width=3),
                legendgroup=ferma_columns_translate.inverse[category_col],
            ),
            row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1
        )
        fig.update_xaxes(title_text=ferma_columns_translate.inverse[num_col], row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1)
        fig.update_yaxes(title_text=ferma_columns_translate.inverse['milk_yield'], row=cetegoty_i // 2 + 1, col=cetegoty_i % 2 + 1)

    title=(f'Диаграммы рассеяния между признаками <b>Удой, кг</b> и <b>{ferma_columns_translate.inverse[num_col]}</b>,' +
           f'<br>разбитые по классам категориальных признаков<br>corr={ferma.milk_yield.corr(ferma[num_col]):.3f}')
    fig.update_layout(
        title=title,
        title_y=0.98,
        title_x=0.5,
        height=1200,
        width=1200,
        legend=dict(
            traceorder='grouped',
            groupclick='toggleitem',
        ),
    )
    fig.show()

- ```energy_unit```

In [30]:
draw_scatter_between_milk_yield_ans('energy_unit', opacity=0.5)

Диаграмма рассеяния выражается квадратичной функцией, однако можно сказать что и линейной тоже. Разбиено по классам заметно только по категории ```Возраст``` (```более_2_лет``` -> больше ```Удой,кг```) и ```порода папа_быка``` (```айдиал``` -> больше ```Удой,кг```). Значение корреляции считается сильным.

- ```raw_protein```

In [31]:
draw_scatter_between_milk_yield_ans('raw_protein', opacity=0.5)

Корреляция считается умеренной, Есть очертание линейной зависимости.

- ```sugar_protein_ratio```

In [32]:
draw_scatter_between_milk_yield_ans('sugar_protein_ratio', opacity=0.3)

Можно наблюдать многомодальное поведение данной переменной, и все же корреляция сильная. Здесь ярко вырожены три группы ($[0.84; 0.86], [0.88; 0.9], [0.93; 0.96]$) => можно рассмотреть возможность перевода признака в категориальный.

- ```fat_content```

In [33]:
draw_scatter_between_milk_yield_ans('fat_content', opacity=0.4)

Значение корреляции хорошее, однако тяжело разглядеть линейную зависимость. Все же видно, что тоже есть группа коров, у которых жирность молока с отрывом больше остальных.

- ```protein_content```

In [34]:
draw_scatter_between_milk_yield_ans('protein_content', opacity=0.3)

Нулевая корреляция и такая же зависимость.

_____
### Вывод:

- Удой (milk_yield) наиболее сильно коррелирует с ```ЭКЕ (Энергетическая кормовая единица)```, ```Возрастом``` и ```Жирностью,%```. Эти признаки показали значимое влияние на целевой признак.
- Мультиколлинеарность обнаружена между возрастом, ```ЭКЕ (Энергетическая кормовая единица)``` и ```Жирностью,%```.
- Для дальнейшего моделирования выбраны признаки: ```ЭКЕ (Энергетическая кормовая единица)```, ```СПО (Сахаро-протеиновое соотношение)```, ```Сырой протеин,г```, ```порода папы_быка```, ```Возраст```, .

_____
_____
## Модели

In [ ]:
from sklearn.feature_selection import RFE


def get_X_y(columns):
    X = ferma[columns]
    y = ferma.milk_yield
    return train_test_split(X, y, random_state=42)


def get_pipepline_reg(X):
    numerical_preprocessor = Pipeline(steps=[('scaler', StandardScaler())])

    categorical_preprocessor = Pipeline(
        steps=[('one_hot', OneHotEncoder(handle_unknown='ignore', drop='first')),
            ('scaler', StandardScaler(with_mean=False))])

    preprocessor = ColumnTransformer(
        [('numerical', numerical_preprocessor, X.select_dtypes(include=[int, float]).columns),
        ('categorical', categorical_preprocessor, X.select_dtypes(include='object').columns)]
    )

    return Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RFE(estimator=LinearRegression(), min_features_to_select=1, step=1, scoring='r2', n_jobs=3, cv=2, verbose=1))]
    )


# from itertools import chain, combinations
# numerical_cols = ferma.drop('milk_yield', axis=1).select_dtypes(include=[int, float]).columns
# categorical_cols = ferma.drop('milk_yield', axis=1).select_dtypes(include='object').columns
# def get_combinations(columns):
#     return chain(*[combinations(columns, i) for i in range(1, len(columns) + 1)])

# # Перебираем все комбинации колонок
# all_combinations = [
#     (num_combo, cat_combo)
#     for num_combo in get_combinations(numerical_cols)
#     for cat_combo in get_combinations(categorical_cols)
# ]

# print(len(all_combinations))
# from sklearn.metrics import r2_score

# data = []
# for i in all_combinations:
#     # X_train, X_test, y_train, y_test = get_X_y(['energy_unit', 'raw_protein', 'breed', 'pasture_type', 'dad_breed', 'age'])
#     X_train, X_test, y_train, y_test = get_X_y([*i[0], *i[1]])

#     pipeline_v1 = get_pipepline_reg(X_train)
#     pipeline_v1.fit(X_train, y_train)
#     r2 = r2_score(y_test, pipeline_v1.predict(X_test))
#     data.append([[*i[0], *i[1]], r2])

In [67]:
X = ferma.drop('milk_yield', axis=1)
y = ferma.milk_yield
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline_v1 = get_pipepline_reg(X_train)
pipeline_v1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['energy_unit', 'raw_protein', 'sugar_protein_ratio', 'fat_content',
       'protein_content'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore')),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  Index(['breed', 'pasture_type', 'dad_breed', 'milk_taste', 'age', 'dad_name'], dtype='object'))])),
                ('model',
                 RFECV(cv=2, estimator=LinearRegression(), n_jobs=3,
                       scoring='r2', verbose=1))])

In [68]:
pipeline_v1.fit(X_train, y_train)

Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['energy_unit', 'raw_protein', 'sugar_protein_ratio', 'fat_content',
       'protein_content'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore')),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  Index(['breed', 'pasture_type', 'dad_breed', 'milk_taste', 'age', 'dad_name'], dtype='object'))])),
                ('model',
                 RFECV(cv=2, estimator=LinearRegression(), n_jobs=3,
                       scoring='r2', verbose=1))])

In [71]:
pipeline_v1.get_feature_names_out()

array(['numerical__energy_unit', 'numerical__raw_protein',
       'numerical__sugar_protein_ratio', 'numerical__fat_content',
       'numerical__protein_content',
       'categorical__pasture_type_холмистое',
       'categorical__milk_taste_не вкусно',
       'categorical__age_менее_2_лет', 'categorical__dad_name_буйный',
       'categorical__dad_name_геркулес'], dtype=object)

In [72]:
pipeline_v1[1].__dict__

{'estimator': LinearRegression(),
 'step': 1,
 'importance_getter': 'auto',
 'cv': 2,
 'scoring': 'r2',
 'verbose': 1,
 'n_jobs': 3,
 'min_features_to_select': 1,
 'n_features_in_': 13,
 'support_': array([ True,  True,  True,  True,  True, False,  True, False,  True,
         True,  True,  True, False]),
 'n_features_': 10,
 'ranking_': array([1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 4]),
 'estimator_': LinearRegression(),
 'cv_results_': {'mean_test_score': array([0.63590866, 0.70493393, 0.75015137, 0.78606397, 0.80218324,
         0.81103311, 0.80782513, 0.80683306, 0.80686398, 0.81303457,
         0.80922628, 0.80989027, 0.81097298]),
  'std_test_score': array([0.02400337, 0.01989089, 0.02862879, 0.0185272 , 0.02325241,
         0.02536394, 0.02583328, 0.02777602, 0.03091167, 0.02874217,
         0.03252233, 0.03359938, 0.0321659 ]),
  'split0_test_score': array([0.65991203, 0.72482481, 0.77878016, 0.80459116, 0.82543565,
         0.83639706, 0.83365841, 0.83460908, 0.83777565, 0.841776

In [62]:
cv_result = pipeline_v1[1].cv_results_

fig = go.Figure()
num_features = np.arange(1, len(cv_result['mean_test_score']))
mean_test_scores = cv_result['mean_test_score']

max_index = np.argmax(mean_test_scores)
max_num_features = num_features[max_index]
max_score = mean_test_scores[max_index]

fig.add_trace(go.Scatter(x=num_features, y=mean_test_scores, mode='lines+markers', name='model_name'))

fig.add_trace(go.Scatter(x=[max_num_features], y=[max_score], mode='markers', marker=dict(color='red', size=10), showlegend=False, name=''))

fig.update_layout(
    title_text='roc_auc моделей в зависимости от числа признаков',
    xaxis_title='Число признаков',
    yaxis_title='r2',
    showlegend=True
)

fig.show()

In [ ]:
# rfecvs = pkl.load(open('pkl/3 feature_search.pkl', 'rb'))

# fig = go.Figure()

# for i, model_name in enumerate(rfecvs):
#     cv_result = pd.DataFrame(rfecvs[model_name]['cv_results_'])
#     num_features = np.arange(10, len(cv_result['mean_test_score']) + 10)
#     mean_test_scores = cv_result['mean_test_score']

#     max_index = np.argmax(mean_test_scores)
#     max_num_features = num_features[max_index]
#     max_score = mean_test_scores[max_index]

#     fig.add_trace(go.Scatter(x=num_features, y=mean_test_scores, mode='lines+markers', name=model_name))

#     fig.add_trace(go.Scatter(x=[max_num_features], y=[max_score], mode='markers', marker=dict(color='red', size=10), showlegend=False, name=''))

# fig.update_layout(
#     title_text='roc_auc моделей в зависимости от числа признаков',
#     xaxis_title='Число признаков',
#     yaxis_title='roc_auc',
#     showlegend=True
# )

# fig.show()

In [ ]:
# pd.DataFrame(data, columns=['param', 'r2']).sort_values(by='r2', ascending=False).iloc[0, 0]

['energy_unit',
 'raw_protein',
 'sugar_protein_ratio',
 'fat_content',
 'protein_content',
 'pasture_type',
 'dad_breed',
 'milk_taste',
 'age',
 'dad_name']

In [ ]:
# pd.DataFrame(data, columns=['param', 'r2']).sort_values(by='r2', ascending=False).iloc[2, 0]

['energy_unit',
 'raw_protein',
 'sugar_protein_ratio',
 'fat_content',
 'protein_content',
 'breed',
 'pasture_type',
 'milk_taste',
 'age',
 'dad_name']

_____
### первая (регрессионная)

In [61]:
X_train, X_test, y_train, y_test = get_X_y(['energy_unit', 'raw_protein', 'sugar_protein_ratio', 'fat_content', 'protein_content', 'pasture_type', 'dad_breed', 'milk_taste', 'age', 'dad_name'])
pipeline_v1 = get_pipepline_reg(X_train)
pipeline_v1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['energy_unit', 'raw_protein', 'sugar_protein_ratio', 'fat_content',
       'protein_content'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore')),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  Index(['pasture_type', 'dad_breed', 'milk_taste', 'age', 'dad_name'], dtype='object'))])),
                ('model', LinearRegression())])

In [ ]:
pipeline_v1.fit(X_train, y_train)
r2_score(y_test, pipeline_v1.predict(X_test))

0.8438192441654442

In [50]:
0.8438192441654442, (('energy_unit', 'raw_protein', 'sugar_protein_ratio', 'fat_content', 'protein_content', 'pasture_type', 'dad_breed', 'milk_taste', 'age', 'dad_name'))

(0.8438192441654442,
 ('energy_unit',
  'raw_protein',
  'sugar_protein_ratio',
  'fat_content',
  'protein_content',
  'pasture_type',
  'dad_breed',
  'milk_taste',
  'age',
  'dad_name'))

In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=2)  # Сохраняем 2 главные компоненты
# X_pca = pca.fit_transform(df[['Возраст', 'ЭКЕ...', 'Жирность,%']])

_____
### втоорая (классификационная)

_____
_____
## Общий вывод

_____
-